# Notebook examples for ZFL/GEE APril/2018

In [1]:
%matplotlib inline

### Principal components

In [2]:
import ee
import IPython.display as disp
from auxil import eepca 
ee.Initialize()

im = ee.Image(
  'LANDSAT/LE07/C01/T1_RT_TOA/LE07_197025_20010626') \
              .select('B1','B2','B3','B4','B5','B7')
    
pcs, lambdas = eepca.pca(im) 

url = pcs.select('pc1','pc2','pc3') \
                .getThumbURL({'min':-0.1,'max':0.1})
disp.Image(url=url)

In [9]:
gdexporttask = ee.batch.Export.image.toDrive(pcs,
               description='driveExportTask', 
               folder='EarthEngineImages',
               fileNamePrefix='PCA_20010626',
               scale=30,
               maxPixels=1e9) 
gdexporttask.start()              

### Canny edge detection

In [ ]:
im = ee.Image(
  'LANDSAT/LE07/C01/T1_RT_TOA/LE07_197025_20010626') \
       .select('B4')
edges = ee.Algorithms.CannyEdgeDetector(im,0.2)

gdexporttask = ee.batch.Export.image.toAsset(edges,
                      description='assetExportTask', 
                      assetId='users/mortcanty/edges',
                      scale=30,
                      maxPixels=1e9) 
gdexporttask.start()   

### HSV Panchromatic sharpening

In [3]:
# Load a Landsat 8 top-of-atmosphere reflectance image.
image = ee.Image('LANDSAT/LC08/C01/T1_TOA/LC08_044034_20140318')

# Convert the RGB bands to the HSV color space.
hsv = image.select(['B4', 'B3', 'B2']).rgbToHsv()

# Swap in the panchromatic band and convert back to RGB.
sharpened = ee.Image.cat([
  hsv.select('hue'), hsv.select('saturation'), image.select('B8')
]).hsvToRgb()

In [4]:
gdexporttask = ee.batch.Export.image.toAsset(sharpened,
                      description='assetExportTask', 
                      assetId='users/mortcanty/pansharpened',
                      scale=30,
                      maxPixels=1e9) 
gdexporttask.start()   